# 时间序列的预处理（Preprocessing of time series）

## 1.导入数据(Import Data)
- 这里我使用的是聚宽的数据库(I use Jukuan database, URL is)网址https://www.joinquant.com/help/api/help?name=JQData
- 为方便后续预测，将数据分开，训练集train_returns用于拟合模型，测试集test_returns用于评估模型拟合的优劣(For analsis later on, the data is divided into train_returns set and test_returns set)
- 测试集只能是很短的几天，因为garch无法预测长期(The test_returns set is short, because GARCH can only predict a few days.)

In [1]:
from jqdatasdk import *
auth('13241809053','809053')
is_auth = is_auth()

auth success 


In [3]:
# 导入上证指数从2015年到2019年的价格，并利用收盘价计算日收益率
# Import the price of Shanghai Stock Index from 2015 to 2019, 
# Use the closing price to calculate the daily rate of return
from jqdatasdk import *
import arch
import numpy as np
import matplotlib.pyplot as plt

auth('13241809053','809053')
is_auth = is_auth()

from jqdatasdk import *
import matplotlib.pyplot as plt
auth('13241809053','809053')
is_auth = is_auth()

price = get_price("000001.XSHG", start_date="2015-1-1", end_date="2019-12-31", frequency='daily')['close']
returns = price.pct_change().dropna()

train_price = get_price("000001.XSHG", start_date="2015-1-1", end_date="2019-12-24", frequency='daily')['close']
train_returns = train_price.pct_change().dropna()

test_price = get_price("000001.XSHG", start_date="2019-12-24", end_date="2019-12-31", frequency='daily')['close']
test_returns = test_price.pct_change().dropna()

train_returns.plot()

## 2. 平稳性检验：ADF单位根检验(Stationary test: ADF unit root test)
- 不存在单位根，说明是平稳序列（No unit root, indicating that it is a stationary sequence）
- H0：序列非平稳（H0: non-stationary sequence）

In [4]:
from arch.unitroot import ADF
print("returns_ADF: ", ADF(train_returns))

returns_ADF:     Augmented Dickey-Fuller Results   
Test Statistic                 -7.066
P-value                         0.000
Lags                               23
-------------------------------------

Trend: Constant
Critical Values: -3.44 (1%), -2.86 (5%), -2.57 (10%)
Null Hypothesis: The process contains a unit root.
Alternative Hypothesis: The process is weakly stationary.


- ADF检验结果，股票收盘价非平稳，收益率平稳（绝大多数都是这种情况）

## 白噪音检验（随机性检验）：LB检验
## (White noise test (randomness test): LB test)

In [5]:
from statsmodels.stats.diagnostic import acorr_ljungbox
import numpy as np
print("returns: ",acorr_ljungbox(train_returns, lags=[6, 12]))

returns:  (array([23.33893119, 39.94053823]), array([6.90556655e-04, 7.35613241e-05]))


- 第一个 array 是 LB 统计量的数值，第二个 array 是 p 值。当 p 值小于显著性水平时认为时序不是白噪声。(The first array is the value of the LB statistic, and the second array is the p-value. When the p-value is less than the significance level, the timing is not considered to be white noise.)
- 测试结果可知，returns不是白噪声，具有分析价值(The test results show that returns are not white noise and have analytical value.)

- 由以上分析可知，股票收益率是非随机的平稳时间序列，适用于AR,MA,ARMA模型，见TS_2.(From the above analysis, it can be known that the stock return rate is a non-random stationary time series, which is suitable for AR, MA, ARMA models, see TS_2)